In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

##### Cargo la base original de todas las peliculas, ya depurada en modulo anterior

In [2]:
# Cargo la base de usuarios y puntajes, que viene del modulo anterior.
df_puntajes = pd.read_csv("df_puntajes.csv")

In [3]:
df_puntajes

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500
...,...,...,...,...
11024284,124380,4.5,1196786159,ns5272
11024285,124380,2.5,1196786030,ns5492
11024286,124380,3.5,1196785679,hs305
11024287,124380,4.5,1196787089,ns7881


In [4]:
df_puntajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11024289 entries, 0 to 11024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  int64  
 3   movieId    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 336.4+ MB


In [5]:
# Reviso valores nulos en columna "userid". Si hay, elimino la fila ya que no me sera util.
print("Valores nulos en columna userid:", df_puntajes["userId"].isnull().sum())

Valores nulos en columna userid: 0


In [6]:
# Controlar filas repetidas.
print("Tamaño del set antes de ejecutar comando para eliminar filas repetidas:", df_puntajes.shape)
df_puntajes.drop_duplicates(inplace=True)			    
print("Tamaño del set despues de ejecutar comando para eliminar filas repetidas:", df_puntajes.shape)

Tamaño del set antes de ejecutar comando para eliminar filas repetidas: (11024289, 4)
Tamaño del set despues de ejecutar comando para eliminar filas repetidas: (11024165, 4)


##### Tomo un df con 1 millon de registros, que se seleccionan al azar (esto me permite reducir tiempo y recursos de cpu).

In [7]:
# Selecciono 1M de registros aleatorios
df_puntajes = df_puntajes.sample(n=1000000, random_state=42)

In [8]:
df_puntajes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 3888695 to 9441547
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   userId     1000000 non-null  int64  
 1   rating     1000000 non-null  float64
 2   timestamp  1000000 non-null  int64  
 3   movieId    1000000 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 38.1+ MB


##### Importo la libreria Surprise

In [9]:
from surprise import Dataset, Reader
from surprise import accuracy, SVD
from surprise.model_selection import train_test_split
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support

In [10]:
# Hago una copia de trabajo del dataframe.
df_muestra = df_puntajes.copy()

In [11]:
# Uso el comando hash() para transformar la columna "item" a numerica y creo otra base para guardar la correlacion.

# Utilizo un diccionario para guardar la correlacion con el codigo original.
diccionario_codigos = {}
for codigo in df_muestra['movieId'].unique():
    codigo_num = hash(codigo) % (10 ** 8)               # Genero el número entero único para cada código de película
    diccionario_codigos[codigo] = codigo_num
    
# Agrego una columna al df con los números correspondientes a cada código de película.
df_muestra['item'] = df_muestra['movieId'].map(diccionario_codigos)

# Creo un dataFrame de correlación entre los códigos de película originales y los códigos numéricos.
df_correlacion = pd.DataFrame({'movieId': list(diccionario_codigos.keys()), 'item': list(diccionario_codigos.values())})

# Imprimimos los resultados
print('DataFrame de puntajes con códigos numéricos:\n', df_muestra) # [['user', 'item', 'rating', 'timestamp']])
print('DataFrame de correlación:\n', df_correlacion)


DataFrame de puntajes con códigos numéricos:
          userId  rating   timestamp movieId      item
3888695   40431     3.0   837504048   ns646  49318095
5020462   51709     5.0  1107206094  ns8734  16389308
8357866  258464     3.0  1182822418  hs3061  54345539
9287538  268333     3.0  1428992622  as5176  80418119
2792891   29030     3.0  1464194046  hs2337  94064760
...         ...     ...         ...     ...       ...
2127804   22103     4.0  1082184112  as6774  99303897
1277326   13112     3.5  1347226475    ds75  65748886
3121097   32561     4.0  1475707038  as6700  96037492
3648687   38014     3.5  1310118323  as1849  15453892
9441547  270071     3.5  1488785537  ns2699  75514247

[1000000 rows x 5 columns]
DataFrame de correlación:
       movieId      item
0       ns646  49318095
1      ns8734  16389308
2      hs3061  54345539
3      as5176  80418119
4      hs2337  94064760
...       ...       ...
22993  ns8212  88219770
22994  ns6168  75697935
22995  as9016  13034490
22996  ns67

In [12]:
# Cambio el orden de las columnas para adaptarlas al standard de la libreria Surprise.
columnas = ['userId', 'item', 'rating', 'timestamp', 'movieId']
df_muestra = df_muestra.reindex(columns=columnas)

# Elimino la columna 'movieId' del original.
df_muestra = df_muestra.drop('movieId', axis=1)

# Renombro para adaptar al standard de la libreria Surprise.
df_muestra = df_muestra.rename(columns={'userId': 'user'})

In [13]:
df_muestra.head()

,user,item,rating,timestamp
3888695,40431,49318095,3.0,837504048
5020462,51709,16389308,5.0,1107206094
8357866,258464,54345539,3.0,1182822418
9287538,268333,80418119,3.0,1428992622
2792891,29030,94064760,3.0,1464194046


In [14]:
df_muestra.to_csv("df_muestra.csv", index=False)

In [15]:
# Ingreso los valores de entrada.
usuario = 51709
pelicula = "ns8734"

In [16]:
# Efectuo la transformacion del codio alfanumerico de pelicula, al codigo numerico de ingreso al modelo.
unique_movieIds = df_correlacion['movieId'].unique()

if pelicula in unique_movieIds:
    peli = df_correlacion.loc[df_correlacion['movieId'] == pelicula, 'item'].iloc[0]
    print(f"El valor de 'item' correspondiente a 'movieId' {pelicula} es: {peli}")
else:
    print(f"El valor de 'movieId' {pelicula} no se encuentra en el DataFrame")

El valor de 'item' correspondiente a 'movieId' ns8734 es: 16389308


In [17]:
df_correlacion.head()

,movieId,item
0,ns646,49318095
1,ns8734,16389308
2,hs3061,54345539
3,as5176,80418119
4,hs2337,94064760


In [18]:
# Creo el objeto Reader y cargo los datos.
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file("./df_muestra.csv", reader=reader)


In [19]:
# Divido los datos en subconjuntos de entrenamiento y prueba.
trainset, testset = train_test_split(data, test_size=0.20)

In [20]:
# Defino modelo SVD y entreno con el conjunto de entrenamiento
model = SVD()
model.fit(trainset)

In [21]:
# Esto es de Rick ahora

# Obtener la predicción del modelo para el usuario y película especificados
pred = model.predict(uid=usuario, iid=peli)

# Obtener la valoración estimada
rating = pred.est

# Si la valoración es mayor o igual a 3, se recomienda la película, de lo contrario, no se recomienda
if rating >= 3.5:
    print("Si se recomienda la película para este usuario.")
else:
    print("No se recomienda la película para este usuario.")


Si se recomienda la película para este usuario.


In [22]:
predictions = model.test(testset)

In [23]:
# Para medir la prediccion, utilizo el RMSE.
accuracy.rmse(predictions)

RMSE: 1.0076


1.0075768882810714

In [24]:
# Metrica MAE.
mae = accuracy.mae(predictions)

MAE:  0.7905


In [25]:
# Calculo la precisión, la exhaustividad y la F1-score
y_true = [int(pred.r_ui >= 3.5) for pred in predictions]    # 1 si el usuario calificó la película con 3.5 o más estrellas, 0 en caso contrario
y_pred = [pred.est >= 3.5 for pred in predictions]          # 1 si el modelo predice una calificación de 3.5 o más estrellas, 0 en caso contrario

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Precision para el modelo es: ", precision)
print("Recall para el modelo es: : ", recall)
print("F1-score para el modelo es: : ", f1)

Precision para el modelo es:  0.7063383858636084
Recall para el modelo es: :  0.6403832637200171
F1-score para el modelo es: :  0.6717457627118644
